In [1]:
import pandas as pd
import wandb
from tqdm.notebook import tqdm
import pickle
from os.path import exists
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math
from matplotlib.ticker import MaxNLocator

from data.get_uci import all_datasets
from analysis.util import fetch, init_uci_dict, get_uci_info

In [2]:
filters = {
    "group": "noise"
}
raw = fetch("soft-gp-2", filters)

100%|██████████| 24/24 [00:09<00:00,  2.65it/s]


In [5]:
uci_info = get_uci_info()

In [12]:
runs = {}
uci_dict = {}
for exp in raw:
    model = exp.config["model.name"]
    dataset = exp.config["dataset.name"]
    num_inducing = exp.config["model.num_inducing"]
    dtype = exp.config["model.dtype"]
    seed = exp.config["training.seed"]
    learn_noise = exp.config["model.learn_noise"]
    train_frac = float(exp.config["dataset.train_frac"])
    uci_dict[(dataset, seed, num_inducing, train_frac, model, learn_noise)] = exp.history
    runs[(dataset, seed, num_inducing, train_frac, model, learn_noise)] = exp.run.id

print(uci_dict.keys())

dict_keys([('houseelectric', 6535, 512, 0.9, 'soft-gp', False), ('houseelectric', 6535, 512, 0.9, 'soft-gp', True), ('buzz', 6535, 512, 0.9, 'soft-gp', False), ('buzz', 6535, 512, 0.9, 'soft-gp', True), ('song', 6535, 512, 0.9, 'soft-gp', False), ('song', 6535, 512, 0.9, 'soft-gp', True), ('3droad', 6535, 512, 0.9, 'soft-gp', False), ('3droad', 6535, 512, 0.9, 'soft-gp', True), ('keggundirected', 6535, 512, 0.9, 'soft-gp', False), ('keggundirected', 6535, 512, 0.9, 'soft-gp', True), ('slice', 6535, 512, 0.9, 'soft-gp', False), ('slice', 6535, 512, 0.9, 'soft-gp', True), ('keggdirected', 6535, 512, 0.9, 'soft-gp', False), ('keggdirected', 6535, 512, 0.9, 'soft-gp', True), ('protein', 6535, 512, 0.9, 'soft-gp', False), ('protein', 6535, 512, 0.9, 'soft-gp', True), ('kin40k', 6535, 512, 0.9, 'soft-gp', False), ('kin40k', 6535, 512, 0.9, 'soft-gp', True), ('bike', 6535, 512, 0.9, 'soft-gp', False), ('bike', 6535, 512, 0.9, 'soft-gp', True), ('elevators', 6535, 512, 0.9, 'soft-gp', False), 

In [14]:
uci_dict[("pol", 6535, 512, 0.9, "soft-gp", True)]

,loss,_step,use_pinv,outputscale,K_zz_norm_inf,K_zz_bin_1e-20,K_zz_bin_0.0,K_zz_bin_1e-10,fit_time,lengthscale,...,_timestamp,K_zz_bin_0.5,K_zz_bin_1e-05,noise,test_nll,inducing_points,test_rmse,K_zz_norm_2,K_zz,K_zz_norm_1
0,33.044548,0,0,0.764951,1.504251,30130,217478,10822,0.032226,0.633354,...,1.725519e+09,512,2812,0.001146,-46.803749,"{'_type': 'histogram', 'values': [55, 54, 100,...",0.253723,17.382153,"{'size': 113991, 'height': 600, 'width': 800, ...",1.504251
1,24.489834,1,0,0.836326,1.636698,28266,220654,9836,0.005442,0.623728,...,1.725519e+09,512,2546,0.001298,-38.283695,None,0.241404,18.987089,None,1.636698
2,19.709291,2,0,0.907574,1.871030,30462,216936,10910,0.005435,0.647622,...,1.725519e+09,512,2946,0.001449,-32.881687,None,0.235290,20.615671,None,1.871030
3,17.436831,3,0,0.976731,2.155188,34198,210564,12716,0.005420,0.683096,...,1.725519e+09,514,3618,0.001602,-29.021215,None,0.231731,22.216261,None,2.155188
4,15.180762,4,0,1.044430,2.466055,38136,203720,14642,0.005446,0.718569,...,1.725519e+09,520,4454,0.001758,-26.153908,None,0.228351,23.794586,None,2.466056
5,13.229930,5,0,1.110033,2.815358,41912,197120,16424,0.005428,0.752302,...,1.725519e+09,524,5298,0.001916,-23.406315,None,0.226208,25.336266,None,2.815357
6,12.323600,6,0,1.171511,3.205022,45760,190356,18146,0.008548,0.784987,...,1.725519e+09,524,6308,0.002082,-20.936817,None,0.224025,26.799126,None,3.205021
7,11.022084,7,0,1.231068,3.569690,48830,184474,19948,0.005476,0.812396,...,1.725519e+09,526,7116,0.002249,-19.342743,None,0.222438,28.225262,None,3.569689
8,10.090011,8,0,1.288306,4.008401,51762,178938,21502,0.005423,0.837430,...,1.725519e+09,530,7978,0.002422,-17.942286,None,0.222075,29.607550,None,4.008401
9,9.295303,9,0,1.344119,4.395619,53692,174820,22840,0.005421,0.857655,...,1.725519e+09,538,8648,0.002599,-16.428429,None,0.221212,30.956362,None,4.395620


In [24]:
UCI_INFO = {
    "N": [int(np.floor(N * 0.9)) for _, N, _, _ in uci_info],
    "D": [D for _, _, D, _ in uci_info],
    # "exact-rep": [e for _, _, _, e in uci_info],
}
KZZ = {}
all_bins = {}

models = ["soft-gp"]

# models = ["sv-gp"]
for ln in [True, False]:
    model = "soft-gp"
    seed = 6535
    frac = 0.9
    num_inducing = 512
    xs = []
    ts = []
    K_zzs = []
    bins1 = []
    bins2 = []
    bins3 = []
    bins4 = []
    bins5 = []
    bins6 = []
    for dataset, _, _, _ in uci_info:
        try:
            xs += [float(uci_dict[(dataset, seed, num_inducing, frac, model, ln)]["test_rmse"][49])]
            ts += [float(np.array(uci_dict[(dataset, seed, num_inducing, frac, model, ln)]["epoch_time"][49]).mean())]
            K_zzs += [uci_dict[(dataset, seed, num_inducing, frac, model, ln)]["K_zz"][i] for i in range(5)]
            bins1 += [uci_dict[(dataset, seed, num_inducing, frac, model, ln)]["K_zz_bin_0.0"][49]]
            bins2 += [uci_dict[(dataset, seed, num_inducing, frac, model, ln)]["K_zz_bin_1e-20"][49]]
            bins3 += [uci_dict[(dataset, seed, num_inducing, frac, model, ln)]["K_zz_bin_1e-10"][49]]
            bins4 += [uci_dict[(dataset, seed, num_inducing, frac, model, ln)]["K_zz_bin_1e-05"][49]]
            bins5 += [uci_dict[(dataset, seed, num_inducing, frac, model, ln)]["K_zz_bin_0.01"][49]]
            bins6 += [uci_dict[(dataset, seed, num_inducing, frac, model, ln)]["K_zz_bin_0.5"][49]]
        except Exception as e:
            xs += [np.nan]
            ts += [np.nan]
            bins1 += [np.nan]
            bins2 += [np.nan]
            bins3 += [np.nan]
            bins4 += [np.nan]
            bins5 += [np.nan]
            bins6 += [np.nan]
            print("Exception", e, model, dataset)

    UCI_INFO[f"{model}-{num_inducing}-{frac}-{seed}-{ln}"] = xs
    # UCI_INFO[f"time-{model}-{num_inducing}-{frac}-{seed}-{ln}"] = ts
    all_bins[f"0.0-{model}-{num_inducing}-{frac}-{seed}-{ln}"] = bins1
    all_bins[f"1e-20-{model}-{num_inducing}-{frac}-{seed}-{ln}"] = bins2
    all_bins[f"1e-10-{model}-{num_inducing}-{frac}-{seed}-{ln}"] = bins3
    all_bins[f"1e-05-{model}-{num_inducing}-{frac}-{seed}-{ln}"] = bins4
    all_bins[f"0.01-{model}-{num_inducing}-{frac}-{seed}-{ln}"] = bins5
    all_bins[f"0.5-{model}-{num_inducing}-{frac}-{seed}-{ln}"] = bins6
    KZZ[f"kzz-{model}-{num_inducing}-{frac}-{seed}-{ln}"] = K_zzs
df = pd.DataFrame(data=UCI_INFO)
df.index = [name.capitalize().replace("_", "-") for name, _, _, _ in uci_info]
df

,N,D,soft-gp-512-0.9-6535-True,soft-gp-512-0.9-6535-False
Pol,13500,26,0.228467,0.189090
Elevators,14939,18,0.389617,0.391146
Bike,15641,17,0.244963,0.207341
Kin40k,36000,8,0.389934,0.242822
Protein,41157,9,0.726131,0.658366
Keggdirected,43944,20,0.081146,0.078714
Slice,48150,385,0.052266,0.050888
Keggundirected,57247,27,0.120411,0.116866
3droad,391386,3,0.727352,0.605115
Song,270000,90,0.797050,0.797475


In [16]:
filters = {
    "group": "ablation"
}
raw2 = fetch("soft-gp-2", filters)

100%|██████████| 218/218 [01:29<00:00,  2.43it/s]


In [21]:
runs = {}
uci_dict2 = {}
for exp in raw2:
    model = exp.config["model.name"]
    dataset = exp.config["dataset.name"]
    num_inducing = exp.config["model.num_inducing"]
    dtype = exp.config["model.dtype"]
    seed = exp.config["training.seed"]
    learn_noise = exp.config["model.learn_noise"]
    train_frac = float(exp.config["dataset.train_frac"])
    batch_size = exp.config["training.batch_size"]
    uci_dict2[(dataset, batch_size, num_inducing, model)] = exp.history
    runs[(dataset, batch_size, num_inducing, model)] = exp.run.id

print(uci_dict2.keys())

dict_keys([('houseelectric', 2048, 256, 'soft-gp'), ('3droad', 2048, 1024, 'soft-gp'), ('3droad', 1024, 1024, 'soft-gp'), ('3droad', 512, 1024, 'soft-gp'), ('3droad', 256, 1024, 'soft-gp'), ('3droad', 4096, 512, 'soft-gp'), ('houseelectric', 1024, 256, 'soft-gp'), ('3droad', 2048, 512, 'soft-gp'), ('3droad', 1024, 512, 'soft-gp'), ('3droad', 512, 512, 'soft-gp'), ('3droad', 256, 512, 'soft-gp'), ('3droad', 4096, 256, 'soft-gp'), ('houseelectric', 512, 256, 'soft-gp'), ('3droad', 2048, 256, 'soft-gp'), ('3droad', 1024, 256, 'soft-gp'), ('3droad', 512, 256, 'soft-gp'), ('3droad', 256, 256, 'soft-gp'), ('keggundirected', 4096, 4096, 'soft-gp'), ('keggundirected', 2048, 4096, 'soft-gp'), ('keggundirected', 1024, 4096, 'soft-gp'), ('keggundirected', 512, 4096, 'soft-gp'), ('keggundirected', 256, 4096, 'soft-gp'), ('keggundirected', 4096, 2048, 'soft-gp'), ('keggundirected', 2048, 2048, 'soft-gp'), ('keggundirected', 1024, 2048, 'soft-gp'), ('keggundirected', 512, 2048, 'soft-gp'), ('keggund

In [29]:
UCI_INFO = {
    "N": [int(np.floor(N * 0.9)) for _, N, _, _ in uci_info],
    "D": [D for _, _, D, _ in uci_info],
    # "exact-rep": [e for _, _, _, e in uci_info],
}
KZZ = {}
all_bins = {}

num_inducings = [256, 512, 1024, 2048, 4096]

# models = ["sv-gp"]
for num_inducing in num_inducings:
    model = "soft-gp"
    batch_size = 1024
    seed = 6535
    frac = 0.9
    xs = []
    ts = []
    K_zzs = []
    bins1 = []
    bins2 = []
    bins3 = []
    bins4 = []
    bins5 = []
    bins6 = []
    for dataset, _, _, _ in uci_info:
        try:
            xs += [float(uci_dict2[(dataset, batch_size, num_inducing, model)]["test_rmse"][49])]
            ts += [float(np.array(uci_dict2[(dataset, batch_size, num_inducing, model)]["epoch_time"][49]).mean())]
            K_zzs += [uci_dict2[(dataset, batch_size, num_inducing, model)]["K_zz"][i] for i in range(5)]
            bins1 += [uci_dict2[(dataset, batch_size, num_inducing, model)]["K_zz_bin_0.0"][49]]
            bins2 += [uci_dict2[(dataset, batch_size, num_inducing, model)]["K_zz_bin_1e-20"][49]]
            bins3 += [uci_dict2[(dataset, batch_size, num_inducing, model)]["K_zz_bin_1e-10"][49]]
            bins4 += [uci_dict2[(dataset, batch_size, num_inducing, model)]["K_zz_bin_1e-05"][49]]
            bins5 += [uci_dict2[(dataset, batch_size, num_inducing, model)]["K_zz_bin_0.01"][49]]
            bins6 += [uci_dict2[(dataset, batch_size, num_inducing, model)]["K_zz_bin_0.5"][49]]
        except Exception as e:
            xs += [np.nan]
            ts += [np.nan]
            bins1 += [np.nan]
            bins2 += [np.nan]
            bins3 += [np.nan]
            bins4 += [np.nan]
            bins5 += [np.nan]
            bins6 += [np.nan]
            print("Exception", e, model, dataset)

    UCI_INFO[f"{model}-{num_inducing}-{batch_size}-{seed}"] = xs
    # UCI_INFO[f"time-{model}-{num_inducing}-{batch_size}-{seed}"] = ts
    all_bins[f"0.0-{model}-{num_inducing}-{batch_size}-{seed}"] = bins1
    all_bins[f"1e-20-{model}-{num_inducing}-{batch_size}-{seed}"] = bins2
    all_bins[f"1e-10-{model}-{num_inducing}-{batch_size}-{seed}"] = bins3
    all_bins[f"1e-05-{model}-{num_inducing}-{batch_size}-{seed}"] = bins4
    all_bins[f"0.01-{model}-{num_inducing}-{batch_size}-{seed}"] = bins5
    all_bins[f"0.5-{model}-{num_inducing}-{batch_size}-{seed}"] = bins6
    KZZ[f"kzz-{model}-{num_inducing}-{batch_size}-{seed}"] = K_zzs
df = pd.DataFrame(data=UCI_INFO)
df.index = [name.capitalize().replace("_", "-") for name, _, _, _ in uci_info]
df

Exception ('song', 1024, 256, 'soft-gp') soft-gp song
Exception ('buzz', 1024, 256, 'soft-gp') soft-gp buzz
Exception ('song', 1024, 512, 'soft-gp') soft-gp song
Exception ('buzz', 1024, 512, 'soft-gp') soft-gp buzz
Exception ('houseelectric', 1024, 512, 'soft-gp') soft-gp houseelectric
Exception ('song', 1024, 1024, 'soft-gp') soft-gp song
Exception ('buzz', 1024, 1024, 'soft-gp') soft-gp buzz
Exception ('houseelectric', 1024, 1024, 'soft-gp') soft-gp houseelectric
Exception ('3droad', 1024, 2048, 'soft-gp') soft-gp 3droad
Exception ('song', 1024, 2048, 'soft-gp') soft-gp song
Exception ('buzz', 1024, 2048, 'soft-gp') soft-gp buzz
Exception ('houseelectric', 1024, 2048, 'soft-gp') soft-gp houseelectric
Exception 'test_rmse' soft-gp slice
Exception ('3droad', 1024, 4096, 'soft-gp') soft-gp 3droad
Exception ('song', 1024, 4096, 'soft-gp') soft-gp song
Exception ('buzz', 1024, 4096, 'soft-gp') soft-gp buzz
Exception ('houseelectric', 1024, 4096, 'soft-gp') soft-gp houseelectric


,N,D,soft-gp-256-1024-6535,soft-gp-512-1024-6535,soft-gp-1024-1024-6535,soft-gp-2048-1024-6535,soft-gp-4096-1024-6535
Pol,13500,26,0.200016,0.189090,0.188740,0.192317,0.191453
Elevators,14939,18,0.383852,0.391154,0.389582,0.388605,0.393405
Bike,15641,17,0.192538,0.207341,0.213648,0.221103,0.221708
Kin40k,36000,8,0.256036,0.242820,0.235801,0.232635,0.233329
Protein,41157,9,0.657544,0.656623,0.656893,0.655590,0.655101
Keggdirected,43944,20,0.079792,0.078811,0.079661,0.079823,0.080119
Slice,48150,385,0.054908,0.051069,0.044625,0.035776,NaN
Keggundirected,57247,27,NaN,0.116632,0.118071,0.115163,0.115035
3droad,391386,3,0.608388,0.605115,0.608126,NaN,NaN
Song,270000,90,NaN,NaN,NaN,NaN,NaN


In [30]:
all_bins.keys()

dict_keys(['0.0-soft-gp-256-1024-6535', '1e-20-soft-gp-256-1024-6535', '1e-10-soft-gp-256-1024-6535', '1e-05-soft-gp-256-1024-6535', '0.01-soft-gp-256-1024-6535', '0.5-soft-gp-256-1024-6535', '0.0-soft-gp-512-1024-6535', '1e-20-soft-gp-512-1024-6535', '1e-10-soft-gp-512-1024-6535', '1e-05-soft-gp-512-1024-6535', '0.01-soft-gp-512-1024-6535', '0.5-soft-gp-512-1024-6535', '0.0-soft-gp-1024-1024-6535', '1e-20-soft-gp-1024-1024-6535', '1e-10-soft-gp-1024-1024-6535', '1e-05-soft-gp-1024-1024-6535', '0.01-soft-gp-1024-1024-6535', '0.5-soft-gp-1024-1024-6535', '0.0-soft-gp-2048-1024-6535', '1e-20-soft-gp-2048-1024-6535', '1e-10-soft-gp-2048-1024-6535', '1e-05-soft-gp-2048-1024-6535', '0.01-soft-gp-2048-1024-6535', '0.5-soft-gp-2048-1024-6535', '0.0-soft-gp-4096-1024-6535', '1e-20-soft-gp-4096-1024-6535', '1e-10-soft-gp-4096-1024-6535', '1e-05-soft-gp-4096-1024-6535', '0.01-soft-gp-4096-1024-6535', '0.5-soft-gp-4096-1024-6535'])

In [37]:
model = "soft-gp"
num_inducing = 512
frac = 0.9
seed = 6535

bin_df = pd.DataFrame()
bin_df[['N', 'D']] = df[['N', 'D']]
# bin_df.index = [name.capitalize().replace("_", "-") for name, _, _, _ in uci_info]
for model_num_inducing in ["soft-gp-256", "soft-gp-512", "soft-gp-1024", "soft-gp-2048", "soft-gp-4096"]:
    num_inducing = int(model_num_inducing.split("-")[-1])
    total = num_inducing**2
    for b in ["0.0", "1e-20", "1e-10", "1e-05", "0.01", "0.5"]:
        bin_df[f"{b}-{model_num_inducing}-{1024}-{seed}"] = np.array(all_bins[f"{b}-{model_num_inducing}-{1024}-{seed}"]) / total
    # bin_df[f"1e-20-{model}-{num_inducing}-{frac}-{seed}"] = np.array(all_bins[f"1e-20-{model}-{num_inducing}-{frac}-{seed}"]) / (512**2)
bin_df = bin_df.sort_values(by=['D'], ascending=[True])
bin_df

,N,D,0.0-soft-gp-256-1024-6535,1e-20-soft-gp-256-1024-6535,1e-10-soft-gp-256-1024-6535,1e-05-soft-gp-256-1024-6535,0.01-soft-gp-256-1024-6535,0.5-soft-gp-256-1024-6535,0.0-soft-gp-512-1024-6535,1e-20-soft-gp-512-1024-6535,...,1e-10-soft-gp-2048-1024-6535,1e-05-soft-gp-2048-1024-6535,0.01-soft-gp-2048-1024-6535,0.5-soft-gp-2048-1024-6535,0.0-soft-gp-4096-1024-6535,1e-20-soft-gp-4096-1024-6535,1e-10-soft-gp-4096-1024-6535,1e-05-soft-gp-4096-1024-6535,0.01-soft-gp-4096-1024-6535,0.5-soft-gp-4096-1024-6535
3droad,391386,3,0.981689,0.005859,0.000519,0.000885,0.000000,0.000000,0.976952,0.006805,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Kin40k,36000,8,0.000000,0.005524,0.160828,0.497559,0.297211,0.038879,0.001221,0.058090,...,0.235273,0.388117,0.262076,0.034887,0.003667,0.086025,0.234257,0.380362,0.260757,0.034931
Protein,41157,9,0.838318,0.101379,0.039612,0.013367,0.001770,0.005554,0.570206,0.196396,...,0.155165,0.097870,0.036341,0.010585,0.457475,0.223007,0.167569,0.103099,0.038256,0.010593
Houseelectric,1844352,11,0.032135,0.062561,0.231537,0.358429,0.204559,0.093994,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bike,15641,17,0.000000,0.000000,0.004944,0.326233,0.533112,0.135712,0.000000,0.000000,...,0.357483,0.454823,0.128623,0.016464,0.000612,0.082232,0.412808,0.392016,0.098284,0.014048
Elevators,14939,18,0.031769,0.035156,0.157379,0.441864,0.314331,0.019501,0.031342,0.085152,...,0.318385,0.400019,0.133342,0.005443,0.012777,0.134340,0.355361,0.371183,0.121375,0.004963
Keggdirected,43944,20,0.121796,0.064941,0.084320,0.215881,0.368500,0.144562,0.109329,0.062233,...,0.152492,0.340286,0.327445,0.057614,0.032457,0.060802,0.150558,0.363400,0.339573,0.053211
Pol,13500,26,0.986603,0.007629,0.001343,0.000519,0.000000,0.003906,0.970215,0.020439,...,0.052964,0.019505,0.004871,0.001285,0.617000,0.220752,0.103178,0.045969,0.011159,0.001943
Keggundirected,57247,27,0.918304,0.047668,0.017273,0.009094,0.002228,0.005432,0.981422,0.009666,...,0.005972,0.002486,0.000901,0.008418,0.933264,0.031194,0.011518,0.011651,0.003529,0.008845
Buzz,524925,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
colors = ['red', 'green', 'blue', 'brown', 'magenta'] * 2
markers = ['x', 'o', '.'] * 2

for dataset in ["3droad", "Kin40k", "Protein", "Houseelectric", "Bike", "Elevators", "Keggdirected", "Pol", "Keggundirected", "Buzz", "Song", "Slice"]:
    plt.figure()
    for model_num_inducing, c, m in zip(["soft-gp-256", "soft-gp-512", "soft-gp-1024", "soft-gp-2048", "soft-gp-4096"], colors, markers):
        bs = ["0.0", "1e-20", "1e-10", "1e-05", "0.01", "0.5"]
        # model_num_inducing = "soft-gp-512"
        num_inducing = int(model_num_inducing.split("-")[-1])
        style = "-" if num_inducing <= 1024 else ":"
        plt.plot(bs, [bin_df[f"{b}-{model_num_inducing}-{1024}-{seed}"][dataset] for b in bs], label=num_inducing, marker=m, color=c, linestyle=style)
    plt.legend()
    plt.title("Inducing")
    plt.ylabel("Proportion")
    plt.xlabel("Kernel Value")
    plt.savefig(f"inducing/{dataset}.png")
    plt.close()